## Spatial model

In [ ]:
import random
import os
import numpy as np
import json
import corner
import random
import os

import pandas as pd
import geopandas as gp
import datetime
import scipy
import matplotlib
import matplotlib.colors as colors
import matplotlib.dates as mdates
import math
import xarray as xr
import emcee
import matplotlib.pyplot as plt

from covid19model.optimization import objective_fcns
from covid19model.models import models
from covid19model.models.utils import name2nis
from covid19model.data import google, sciensano, polymod, model_parameters
from covid19model.visualization.output import population_status, infected
from covid19model.visualization.optimization import plot_fit, traceplot

# OPTIONAL: Load the "autoreload" extension so that package code can change
%load_ext autoreload
# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

### Check name2nis functionality

In [ ]:
name2nis('Arrondissement Roeselare')

In [ ]:
name2nis('arrondissement Roeselare')

In [ ]:
name2nis('Arrondissement oeselare')

In [ ]:
name2nis(5)

### Extract population size (initN)

In [ ]:
initN_df=pd.read_csv('../../data/interim/census_2011/initN.csv', index_col=[0])
initN=initN_df.iloc[:,2:].values

In [ ]:
initN_df.head()

### Load Polymod interaction matrices

In [ ]:
dummy_initN, Nc_home, Nc_work, Nc_schools, Nc_transport, Nc_leisure, Nc_others, Nc_total = polymod.get_interaction_matrices()

### Initialize model

In [ ]:
# Load the parameters using `get_COVID19_SEIRD_parameters()`.
params = model_parameters.get_COVID19_SEIRD_parameters(spatial=True)
# Add the delayed ramp parameters to the parameter dictionary.
params.update({'l': 5,
              'tau': 5}) 
# Define a cluster of 5 infectees in one or two arrondissements
E = np.zeros(initN.shape)
E[np.where(initN_df.index.values==name2nis('arrondissement ieper'))[0][0],:] = 2 
E[np.where(initN_df.index.values==name2nis('arrondissement tongeren'))[0][0],:] = 2
# Define the initial condition: one exposed inidividual in every age category
initial_states = {'S': initN, 'E': E}
# Load the compliance model
from covid19model.models.compliance import ramp_2
# Initialize the model
model = models.COVID19_SEIRD_sto_spatial(initial_states, params, compliance=ramp_2, discrete=True)

### Change beta to a higher value

In [ ]:
model.parameters['beta'] = 0.5

### Define a checkpoints dictionary and perform some simulations

In [ ]:
chk = {
    'time': ['20-04-2020'],
    'Nc':   [0.3*(1.0*Nc_home + 0.3*Nc_work + 0.4*Nc_transport)]
}

In [ ]:
fig,ax=plt.subplots()
for i in range(20):
    out=model.sim('21-09-2020',excess_time=50,checkpoints=chk)
    sumNIS=out.sum(dim="NIS").sum(dim="Nc")
    plt.plot(out["time"].values[50:200],sumNIS["H_in"].values[50:200],alpha=0.05,color='green')

## Visualisation on a map

### Read the arrondissements shape file

In [ ]:
# Read shape file
gemeentes = gp.read_file("../../data/raw/geopandas/BE/BELGIUM__Municipalities.shp")
# select the columns that you with to use for the dissolve and that will be retained
gemeentes_new = gemeentes[['arrond','geometry']]
# dissolve the state boundary by region 
arrondissementen = gemeentes_new.dissolve(by='arrond')

### Perform a single simulation

In [ ]:
out=model.sim(250,checkpoints=chk)
sumNIS=out.sum(dim="NIS").sum(dim="Nc")
plt.plot(out["time"],sumNIS["M"],alpha=0.15,color='green')

In [ ]:
output_path = '../results/maps'

# create the plot
fig, ax = plt.subplots(figsize = (12,12))

start = 0

for day in range(start,len(out['time'].values)):
    # Get data
    data2plot = out.sum(dim="Nc")['M'][:,day].values/initN_df['total'].values*100
    arrondissementen['data'] = data2plot
    # Visualize data
    if day == start:
        fig = arrondissementen.plot(column = 'data', ax=ax, cmap='plasma',
                                    norm=colors.LogNorm(vmin=0.001, vmax=1), legend=True, edgecolor = 'k')
    else:
        fig = arrondissementen.plot(column = 'data', ax=ax, cmap='plasma',
                                    norm=colors.LogNorm(vmin=0.001, vmax=1), legend=False, edgecolor = 'k')
    # Disable axis
    ax.set_axis_off()
    # This will save the figure as a high-res png in the output path. you can also save as svg if you prefer.
    chart = fig.get_figure()
    chart.savefig('maps/'+str(day)+'_B.jpg',dpi=100)
    
    plt.close()

In [ ]:
fig,ax=plt.subplots()
fig.set_size_inches(16, 4)
ax.plot(out['time'][50:200],out.sel(NIS=name2nis('arrondissement antwerpen')).sum(dim='Nc')['M'][50:200],color='blue',alpha=0.40)
ax.plot(out['time'][50:200],out.sel(NIS=name2nis('arrondissement brussel-hoofdstad')).sum(dim='Nc')['M'][50:200],color='red',alpha=0.40)
ax.plot(out['time'][50:200],out.sel(NIS=name2nis('arrondissement luik')).sum(dim='Nc')['M'][50:200],color='black',alpha=0.40)
ax.legend(['Antwerp','Brussels','Luik'])
ax.axvline(130,color='black',linestyle='dashed')
ax.set_title('Daily hospitalizations $(H_{in})$')
fig.savefig('daily_hosp.jpg',dpi=300)